In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q gradio
!pip install -q efficientnet-pytorch

import gradio as gr
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━

In [6]:
!pip install -q timm
import timm

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, x):
        batch_size, seq_len, d_model = x.size()

        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)

        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        output = self.W_o(context)
        return output

In [4]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNet, self).__init__()
        try:
            self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
            print("Tải trọng số pretrained thành công!")
        except Exception as e:
            print(f"Lỗi khi tải trọng số pretrained: {e}")
            self.base_model = timm.create_model('efficientnet_b0', pretrained=False)
            print("Khởi tạo mô hình với trọng số ngẫu nhiên.")

        self.conv_stem = self.base_model.conv_stem
        self.bn1 = self.base_model.bn1
        self.blocks = self.base_model.blocks
        self.conv_head = self.base_model.conv_head
        self.bn2 = self.base_model.bn2
        self.global_pool = self.base_model.global_pool

        # Lấy số kênh đầu ra từ các khối bằng cách forward một tensor mẫu
        self.feature_dims = []
        self.feature_sizes = []
        with torch.no_grad():
            x = torch.randn(1, 3, 224, 224)
            x = self.conv_stem(x)
            x = self.bn1(x)
            for block in self.blocks:
                x = block(x)
                self.feature_dims.append(x.size(1))  # Số kênh
                self.feature_sizes.append(x.size(2))  # Kích thước không gian (H)

        # Debug: In số kênh và kích thước không gian
        print("Số kênh của các khối:", self.feature_dims)
        print("Kích thước không gian của các khối:", self.feature_sizes)

        # Tầng convolution để chuẩn hóa đặc trưng (lấy 3 khối cuối)
        self.fusion_convs = nn.ModuleList([
            nn.Conv2d(dim, 320, kernel_size=1) for dim in self.feature_dims[-3:]
        ])

        # Tầng pooling để chuẩn hóa kích thước không gian về 7x7
        self.spatial_norm = nn.ModuleList([
            nn.AdaptiveMaxPool2d(output_size=7) for _ in range(3)
        ])

        # SE Block cho hợp nhất
        self.se_block = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(320 * 3, 320 // 8, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(320 // 8, 320 * 3, kernel_size=1),
            nn.Sigmoid()
        )

        # MHA
        self.mha = MultiHeadAttention(d_model=320 * 3, num_heads=8)

        # Tầng fully connected
        self.fc = nn.Linear(320 * 3, num_classes)

        # Đóng băng backbone
        for name, param in self.base_model.named_parameters():
            if "blocks.5" not in name:
                param.requires_grad = False

    def forward(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)

        # Lấy đặc trưng từ các khối
        features = []
        for i, block in enumerate(self.blocks):
            x = block(x)
            if i >= len(self.blocks) - 3:  # Lấy 3 khối cuối
                features.append(x)

        # Chuẩn hóa kích thước không gian và số kênh
        fused_features = []
        for feat, conv, norm in zip(features, self.fusion_convs, self.spatial_norm):
            feat = norm(feat)  # Chuẩn hóa kích thước không gian về 7x7
            feat = conv(feat)  # Chuẩn hóa số kênh về 320
            fused_features.append(feat)

        fused = torch.cat(fused_features, dim=1)  # [batch_size, 320*3, 7, 7]
        se_weights = self.se_block(fused)
        fused = fused * se_weights  # Áp dụng SE
        fused = self.global_pool(fused).squeeze(-1).squeeze(-1)  # [batch_size, 320*3]

        # Áp dụng MHA
        fused = fused.unsqueeze(1)  # [batch_size, 1, 320*3]
        fused = self.mha(fused)
        fused = fused.squeeze(1)  # [batch_size, 320*3]

        # Tầng fully connected
        x = self.fc(fused)
        return x

In [8]:
# Danh sách lớp COCO 2017 (80 lớp)
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# Biến đổi ảnh đầu vào
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Hàm dự đoán ảnh
def predict_image(image, model, device, threshold=0.5):
    # Chuyển ảnh sang PIL Image nếu là numpy array (từ Gradio)
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image).convert('RGB')

    # Áp dụng biến đổi và chuyển sang tensor
    image_tensor = val_transform(image).unsqueeze(0).to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probs = torch.sigmoid(output).squeeze().cpu().numpy()

    # Nhãn vượt ngưỡng
    predicted_labels = [(COCO_CLASSES[i], float(probs[i])) for i in range(len(probs)) if probs[i] > threshold]
    predicted_labels = sorted(predicted_labels, key=lambda x: x[1], reverse=True)

    # Top-5 nhãn có xác suất cao nhất
    top5_indices = np.argsort(probs)[-5:][::-1]
    top5_labels = [(COCO_CLASSES[i], float(probs[i])) for i in top5_indices]

    return image, predicted_labels, top5_labels

# Hàm giao diện Gradio
def gradio_predict(image, threshold):
    if image is None:
        return None, "Vui lòng tải lên một ảnh!", None

    # Dự đoán
    image, predicted_labels, top5_labels = predict_image(image, model, device, threshold)

    # Tạo output text cho nhãn vượt ngưỡng
    if predicted_labels:
        output_text = "Nhãn dự đoán (xác suất > {}):\n".format(threshold)
        for label, prob in predicted_labels:
            output_text += f"{label}: {prob:.4f}\n"
    else:
        output_text = f"Không có nhãn nào được dự đoán với xác suất > {threshold}."

    # Tạo bảng top-5 nhãn
    top5_df = pd.DataFrame(top5_labels, columns=["Nhãn", "Xác suất"])

    return image, output_text, top5_df

# Xác định thiết bị (GPU hoặc CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Tải mô hình từ checkpoint
model = CustomEfficientNet(num_classes=80).to(device)
checkpoint_path = "/content/drive/MyDrive/COCO2017/model_best.pth"

try:
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Đã tải trạng thái mô hình từ checkpoint (định dạng dictionary)!")
    else:
        model.load_state_dict(checkpoint)
        print("Đã tải trạng thái mô hình từ checkpoint (định dạng state_dict trực tiếp)!")
    model.eval()
except Exception as e:
    print(f"Lỗi khi tải mô hình: {e}")
    raise

# Tạo giao diện Gradio
iface = gr.Interface(
    fn=gradio_predict,
    inputs=[
        gr.Image(type="numpy", label="Tải ảnh lên"),
        gr.Slider(minimum=0.1, maximum=0.9, value=0.5, step=0.05, label="Ngưỡng xác suất")
    ],
    outputs=[
        gr.Image(type="pil", label="Ảnh đầu vào"),
        gr.Textbox(label="Nhãn dự đoán"),
        gr.Dataframe(label="Top-5 nhãn có xác suất cao nhất")
    ],
    title="Ứng dụng gắn nhãn ảnh đa nhãn (COCO 2017)",
    description="Tải lên một ảnh để gắn nhãn từ 80 lớp COCO 2017. Điều chỉnh ngưỡng xác suất để kiểm soát số lượng nhãn. Top-5 nhãn có xác suất cao nhất được hiển thị để hỗ trợ kiểm tra."
)

# Chạy giao diện
iface.launch(share=True)

Sử dụng thiết bị: cpu
Tải trọng số pretrained thành công!
Số kênh của các khối: [16, 24, 40, 80, 112, 192, 320]
Kích thước không gian của các khối: [112, 56, 28, 14, 14, 7, 7]
Đã tải trạng thái mô hình từ checkpoint (định dạng state_dict trực tiếp)!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92b13a682831952d75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
